In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
import os

## export setting

In [ ]:
strDate = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
icd_version = "10"
output_path = ""
input_path = "icd10/icd10_title_vector.csv"


In [ ]:
if icd_version == "10":
    output_path = "icd10_result"
else:
    output_path = "icd9_result"

if os.path.isdir(output_path) == False:
    os.mkdir(output_path)

output_path = output_path+"/"+strDate

if os.path.isdir(output_path) == False:
    os.mkdir(output_path)

In [ ]:
df_ori = pd.read_csv(input_path)
df = df_ori 
df.columns


In [ ]:
df.head()

## check outlier by scatterplot 

In [ ]:
df["disease_type"] = 0

df["disease_type"] = df["disease_type"].astype(int)

dg = df.groupby('disease')

lstDiseaseT = []

for group_key, group_value in dg:
    if group_key not in lstDiseaseT:
        lstDiseaseT.append(group_key)
    df.loc[(df.disease == group_key),['disease_type']] = lstDiseaseT.index(group_key)

print(lstDiseaseT) 


In [ ]:
df.head()

In [ ]:
df.loc[(df.disease_type == 1),:].head()

In [ ]:
sns.set(rc = {'figure.figsize':(25,10)})
g = sns.scatterplot(x = df.vector ,y = df.disease_type, hue=df.disease)
strFileName = output_path+"/icd"+icd_version+"_outlier.jpeg"
plt.savefig(strFileName, dpi = 300)


## calculate: avg,median,std,max value,min value,std count,std count(avg)

In [ ]:
df["AVG"]=0
df["MEDIAN"]=0
df["STD"]=0
df["MAX"]=0
df["MIN"]=0
 
for strD in lstDiseaseT:
    max = float(df.loc[(df.disease==strD),["vector"]].max())
    min = float(df.loc[(df.disease==strD),["vector"]].min())
    avg = float(df.loc[(df.disease==strD),["vector"]].mean())
    med = float(df.loc[(df.disease==strD),["vector"]].median())
    std = float(df.loc[(df.disease==strD),["vector"]].std())

    df.loc[(df.disease == strD),['MAX']] = max
    df.loc[(df.disease == strD),['MIN']] = min
    df.loc[(df.disease == strD),['AVG']] = avg
    df.loc[(df.disease == strD),['MEDIAN']] = med
    df.loc[(df.disease == strD),['STD']] = std


df.head()


In [ ]:
df["STD_COUNT"] = 0
for idx in df.index:
    df.iloc[idx,df.columns.get_loc('STD_COUNT')] = abs((df.iloc[idx].AVG - df.iloc[idx].vector) / df.iloc[idx].STD)
    

In [ ]:
df["STD_COUNT(AVG)"] = 0
for strD in lstDiseaseT:
    df.loc[(df.disease == strD),['STD_COUNT(AVG)']] = float(df.loc[(df.disease==strD),["STD_COUNT"]].mean())

In [ ]:
# df.drop(columns=["disease_type"],inplace =True)
# df.head()

## find the max STD_COUNT by disease

In [ ]:
df_all_join = None

lstJoin = []
for i in range(0,len(lstDiseaseT)):
    strD = lstDiseaseT[i]
    lstJoin.append(df.loc[(df.disease == strD),:].sort_values("STD_COUNT",ascending=0))


df_all_join = pd.concat(lstJoin)

## export to csv

In [ ]:
strFileName = output_path+"/icd"+icd_version+"_with_outlier.csv"
df_all_join.to_csv(strFileName)

## remove outlier
1.STD COUNT over than 3

In [ ]:
df.loc[(df.STD_COUNT>= 3),["disease","icd_code","title","STD_COUNT"]]

In [ ]:
strFileName = output_path+"/icd"+icd_version+"_over_" + str(3) + "_STD_.csv"
df.loc[(df.STD_COUNT>= 3),["disease","icd_code","title","STD_COUNT"]].to_csv(strFileName)

lst = df.loc[(df.STD_COUNT>= 3),["disease","icd_code","title","STD_COUNT"]].index

if len(lst):          
    df.drop(lst,inplace=True)


In [ ]:
df.loc[(df.STD_COUNT>= 3),["disease","icd_code","title","STD_COUNT"]]

2.vector is 0 

In [ ]:
df.loc[(df.vector==0),["disease","icd_code","title","vector"]]

In [ ]:
strFileName = output_path+"/icd"+icd_version+"_vec_is_0.csv"
df.loc[(df.vector==0),["disease","icd_code","title","vector"]].to_csv(strFileName)

df.drop(df.loc[(df.vector==0),["vector"]].index,inplace=True)

In [ ]:
df.loc[(df.vector==0),["disease","icd_code","title","vector"]]

In [ ]:
sns.set(rc = {'figure.figsize':(25,10)})
g = sns.scatterplot(x = df.vector ,y = df.disease_type, hue=df.disease)
strFileName = output_path+"/icd"+icd_version+"_remove_outlier.jpeg"
plt.savefig(strFileName, dpi = 300)

## compare vector and get intersection count

In [ ]:
strD1 = "psychosis"
strD2 = "heart_type_disease"
strD3 = "neurological_type_disease"
df

In [ ]:
def getCrossRef(aryNp1,aryNp2):

    lst = []
    lstAvg = []

    #print(len(dic1),len(dic2))

    for v1 in aryNp1:
        lst = []
        for v2 in aryNp2:

            lst.append([abs(v1-v2)])

        lstAvg.append(np.array(lst).mean())

    return np.array(lstAvg).mean()

In [ ]:
for strD in lstDiseaseT:
    avg = float(df.loc[(df.disease==strD),["vector"]].mean())
    med = float(df.loc[(df.disease==strD),["vector"]].median())

    df.loc[(df.disease == strD),['AVG']] = avg
    df.loc[(df.disease == strD),['MEDIAN']] = med

lstJoin = []
lstJoin.append(df.loc[(df.disease == strD1),["disease","icd_code","title","vector","AVG","MEDIAN"]])
lstJoin.append(df.loc[(df.disease == strD2),["disease","icd_code","title","vector","AVG","MEDIAN"]])
lstJoin.append(df.loc[(df.disease == strD3),["disease","icd_code","title","vector","AVG","MEDIAN"]])
dfCompare = pd.concat(lstJoin)

strFileName = output_path+"/compare_disease_list.csv"
dfCompare.to_csv(strFileName)


In [ ]:
dfD1 = df.loc[(df.disease == strD1),["vector"]]
dfD2 = df.loc[(df.disease == strD2),["vector"]]
dfD3 = df.loc[(df.disease == strD3),["vector"]]

lstsubVecDisease = [
    strD1+" - "+strD2,
    strD1+" - "+strD3,
    strD2+" - "+strD1,
    strD2+" - "+strD3,
    strD3+" - "+strD1,
    strD3+" - "+strD2]

lstCorossRef = []
lstCorossRef.append(getCrossRef(np.array(dfD1),np.array(dfD2)))
lstCorossRef.append(getCrossRef(np.array(dfD1),np.array(dfD3)))
lstCorossRef.append(getCrossRef(np.array(dfD2),np.array(dfD1)))
lstCorossRef.append(getCrossRef(np.array(dfD2),np.array(dfD3)))
lstCorossRef.append(getCrossRef(np.array(dfD3),np.array(dfD1)))
lstCorossRef.append(getCrossRef(np.array(dfD3),np.array(dfD2)))

lstAVGSubVec = []
lstAVGSubVec.append(abs(dfD1.vector.mean()-dfD2.vector.mean()))
lstAVGSubVec.append(abs(dfD1.vector.mean()-dfD3.vector.mean()))
lstAVGSubVec.append(abs(dfD2.vector.mean()-dfD1.vector.mean()))
lstAVGSubVec.append(abs(dfD2.vector.mean()-dfD3.vector.mean()))
lstAVGSubVec.append(abs(dfD3.vector.mean()-dfD1.vector.mean()))
lstAVGSubVec.append(abs(dfD3.vector.mean()-dfD2.vector.mean()))

lstMedSubVec = []
lstMedSubVec.append(abs(dfD1.vector.median()-dfD2.vector.median()))
lstMedSubVec.append(abs(dfD1.vector.median()-dfD3.vector.median()))
lstMedSubVec.append(abs(dfD2.vector.median()-dfD1.vector.median()))
lstMedSubVec.append(abs(dfD2.vector.median()-dfD3.vector.median()))
lstMedSubVec.append(abs(dfD3.vector.median()-dfD1.vector.median()))
lstMedSubVec.append(abs(dfD3.vector.median()-dfD2.vector.median()))

strFileName = output_path+"/icd"+icd_version+"_sub_vec.csv"
d = {"case":lstsubVecDisease,"cross_ref":lstCorossRef,
    "avg_sub_vec":lstAVGSubVec,"median_sub_vec":lstMedSubVec}
pd.DataFrame(data=d).to_csv(strFileName)


In [ ]:
def getConnString(path:str = './db_conn.txt')->dict:
    dic_conn_str = {}

    f = open('./db_conn.txt', 'r')

    lst_tmp = []
    for line in f.readlines():
        lst_tmp = line.split(':')
        dic_conn_str[lst_tmp[0]] = lst_tmp[1].replace('\n','')

    f.close()
    del lst_tmp

    print(dic_conn_str)
    return dic_conn_str

In [ ]:
import psycopg2


dic = getConnString(path='./db_conn.txt')

conn_string = f"host={dic['host']} dbname={dic['dbname']} user={dic['user']} password={dic['password']}"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

def excuteSqltoInt(cursor,strSql):
    lst = []

    cursor.execute(strSql)
    for row in cursor:
       lst = lst + list(row)
    return lst

def getCountByIcdCode(cursor,strIcdCodeList,strVer):
	intRtn = 0
	strSql = """
    select count(*) as count from
	(
		select 
			distinct subject_id
		from 
			mimic_hosp.diagnoses_icd 
		where 
			icd_code in("""+strIcdCodeList+""")
		and
			icd_version = '"""+strVer+"""'
	) as tmp
    """

	lst = excuteSqltoInt(cursor,strSql)
	intRtn = int(lst[0])
	print(strSql)

	return 	intRtn

def getIntersectionCountByIcdCode(cursor,strIcdCodeList1,strIcdCodeList2,strVer):
	intRtn = 0
	strSql = """
	select count(*) as count from 
	(
		select 
			distinct subject_id
		from 
			mimic_hosp.diagnoses_icd 
		where 
			icd_code in("""+strIcdCodeList1+""")
		and
			icd_version = '"""+strVer+"""'
	) as a,
	(
		select 
			distinct subject_id
		from 
			mimic_hosp.diagnoses_icd 
		where 
			icd_code in("""+strIcdCodeList2+""")
		and
			icd_version = '"""+strVer+"""'
	) as b
	where a.subject_id = b.subject_id
    """

	lst = excuteSqltoInt(cursor,strSql)
	intRtn = int(lst[0])
	print(strSql)
	
	return intRtn

In [ ]:
def getIcdCodeList(dfIcdCode):

    strIcdCodeList = ""
    
    for idx in dfIcdCode.index:
        strIcdCodeList = strIcdCodeList + "'" + str(dfIcdCode.loc[idx].icd_code) + "',"
	
    print(strIcdCodeList)
    return strIcdCodeList[0:len(strIcdCodeList)-1]

In [ ]:
dfIcdCode1 = df.loc[(df.disease == strD1),["icd_code"]]
dfIcdCode2 = df.loc[(df.disease == strD2),["icd_code"]]
dfIcdCode3 = df.loc[(df.disease == strD3),["icd_code"]]

if len(dfIcdCode1) == 0 | len(dfIcdCode2) == 0 | len(dfIcdCode3) == 0 :
    print(strD1 + " icd_code count:",len(dfIcdCode1))
    print(strD2 + " icd_code count:",len(dfIcdCode2))
    print(strD3 + " icd_code count:",len(dfIcdCode3))
else:
    lstCodeList=[]
    lstCodeList.append(getIcdCodeList(dfIcdCode1))
    lstCodeList.append(getIcdCodeList(dfIcdCode2))
    lstCodeList.append(getIcdCodeList(dfIcdCode3))

    lstDisease=[]
    lstDisease.append(strD1)
    lstDisease.append(strD2)
    lstDisease.append(strD3)

    lstCount=[]
    lstCount.append(getCountByIcdCode(cursor,lstCodeList[0],icd_version))
    lstCount.append(getCountByIcdCode(cursor,lstCodeList[1],icd_version))
    lstCount.append(getCountByIcdCode(cursor,lstCodeList[2],icd_version))

    lstIntersectionCase=[]
    lstIntersectionCase.append(strD1 + "__intersection__" + strD2)
    lstIntersectionCase.append(strD1 + "__intersection__" + strD3)
    lstIntersectionCase.append(strD2 + "__intersection__" + strD3)

    lstIntersectionCount=[]
    lstIntersectionCount.append(getIntersectionCountByIcdCode(cursor,lstCodeList[0],lstCodeList[1],icd_version))
    lstIntersectionCount.append(getIntersectionCountByIcdCode(cursor,lstCodeList[0],lstCodeList[2],icd_version))
    lstIntersectionCount.append(getIntersectionCountByIcdCode(cursor,lstCodeList[1],lstCodeList[2],icd_version))

    print(len(lstDisease),len(lstCount),len(lstIntersectionCase),len(lstIntersectionCount))
    
    strFileName = output_path+"/icd"+icd_version+"_subjectId_count.csv"
    d = {
        "disease":lstDisease,
        "count":lstCount,
        "intersection_case":lstIntersectionCase,
        "intersection_count":lstIntersectionCount
        }
    pd.DataFrame(data=d).to_csv(strFileName)